# 5維追強因子

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

from plotly.subplots import make_subplots
import plotly.graph_objects as go

/Users/tp_mini/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [49]:
def reinforce_signal(signal_series):

    return signal_series.rolling(window=20, min_periods=1).apply(lambda x: -1 if (x == -1).sum() >= 2 else 1, raw=False)


def get_5dimen_data(returns, strat_signal='signal'):

    returns["pct_power_5"] = np.power(returns.Close, 5)
    tajm = 242
    alpha = 2 / (tajm + 1)
    returns['cumsum_bm'] = returns.Close.cumsum()
    returns['ma_cumsum_bm'] = returns.cumsum_bm.rolling(60).mean()
    returns['pct_for_calculating_profit'] = returns.Close.shift(-2)

    returns["pct_power_5_ema"] = returns["pct_power_5"].ewm(alpha=alpha, adjust=False).mean()
    returns['pct_power_5_ema_shift_1'] = returns.pct_power_5_ema.shift(1)

    returns['signal'] = returns.apply(lambda row: 1 if row['pct_power_5_ema'] > row['pct_power_5_ema_shift_1']*1.01 or row['pct_power_5_ema'] > 0 else -1, axis=1)

    returns['signal_reinforced'] = reinforce_signal(returns['signal'])


    returns['strat_pct'] = returns[strat_signal] * returns.pct_for_calculating_profit
    
    
    return returns

def plot_through_5dimen(returns, symbol, strat_signal):
        
    k = 0
    u = -1

    fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=symbol)

    fig.add_trace(go.Scatter(
        x=returns.index[k:u],
        y=returns.iloc[k:u].cumsum_bm,
        mode='lines',
        name='Close Price'
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=returns.index[k:u],
        y=returns.iloc[k:u]['strat_pct'].cumsum(),
        mode='lines',
        name='Cumulative Strategy %'
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        x=returns.index[k:u],
        y=returns.iloc[k:u]["signal"],
        mode='lines',
        name='Signal'
    ), row=2, col=1)
    fig.add_trace(go.Scatter(
        x=returns.index[k:u],
        y=returns.iloc[k:u]["signal_reinforced"],
        mode='lines',
        name='Signal'
    ), row=3, col=1)

    fig.update_layout(
        title=symbol,
        template="plotly_white",
        height=600
    )

    fig.update_xaxes(title_text="Index", row=2, col=1)
    fig.update_yaxes(title_text="Signal", row=2, col=1)

    fig.show()

In [58]:
strat_signal='signal_reinforced'
symbol = "1234.TW"
data = yf.download(symbol, start='2018-01-01')
# data = yf.download(symbol, interval='1h')
returns = data[["Close"]].pct_change().dropna()
returns = get_5dimen_data(returns, strat_signal)


plot_through_5dimen(returns, symbol, strat_signal)


[*********************100%***********************]  1 of 1 completed


In [10]:
returns['signal_reinforced'] = returns.signal.apply()

,Close,pct_power_5,cumsum_bm,ma_cumsum_bm,pct_for_calculating_profit,pct_power_5_ema,pct_power_5_ema_shift_1,signal,strat_pct
Datetime,,,,,,,,,
2023-01-30 10:30:00-05:00,0.002233,5.550881e-14,0.002233,NaN,-0.010568,5.550881e-14,NaN,1,-0.010568
2023-01-30 11:30:00-05:00,0.000025,9.474041e-24,0.002258,NaN,-0.010505,5.505195e-14,5.550881e-14,1,-0.010505
2023-01-30 12:30:00-05:00,-0.010568,-1.318317e-10,-0.008311,NaN,-0.008493,-1.030435e-12,5.505195e-14,-1,0.008493
2023-01-30 13:30:00-05:00,-0.010505,-1.279595e-10,-0.018816,NaN,-0.007853,-2.075119e-12,-1.030435e-12,-1,0.007853
2023-01-30 14:30:00-05:00,-0.008493,-4.420102e-11,-0.027309,NaN,0.014210,-2.421834e-12,-2.075119e-12,-1,-0.014210
...,...,...,...,...,...,...,...,...,...
2025-01-27 11:30:00-05:00,-0.005323,-4.271940e-12,1.130108,1.174020,0.006362,3.977126e-08,4.010134e-08,1,0.006362
2025-01-27 12:30:00-05:00,-0.008061,-3.402740e-11,1.122048,1.173706,-0.004944,3.944364e-08,3.977126e-08,1,-0.004944
2025-01-27 13:30:00-05:00,0.006362,1.042591e-11,1.128410,1.173118,0.006489,3.911909e-08,3.944364e-08,1,0.006489
